In [1]:
import asyncio
import websockets
import json
import logging
from jetbot import Robot, Camera, bgr8_to_jpeg
import base64
import time  # Import the 'time' module

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# 설정
WEBSOCKET_PORT = 8766
CAMERA_WIDTH = 300
CAMERA_HEIGHT = 300

# JetBot 초기화
try:
    robot = Robot()
    camera = Camera.instance(width=CAMERA_WIDTH, height=CAMERA_HEIGHT)
    logger.info("JetBot initialized successfully.")
except Exception as e:
    logger.error(f"JetBot initialization failed: {e}")
    robot = None
    camera = None

# 명령 처리
async def handle_command(command, parameters=None):
    parameters = parameters or {}
    if not robot:
        logger.error("Robot not initialized.")
        return False

    try:
        if command == "forward":
            robot.forward(parameters.get("speed", 0.4))
            await asyncio.sleep(parameters.get("duration", 1.0))
            robot.stop()
        elif command == "backward":
            robot.backward(parameters.get("speed", 0.4))
            await asyncio.sleep(parameters.get("duration", 1.0))
            robot.stop()
        elif command == "left":
            robot.left(parameters.get("speed", 0.3))
            await asyncio.sleep(parameters.get("duration", 0.7))
            robot.stop()
        elif command == "right":
            robot.right(parameters.get("speed", 0.3))
            await asyncio.sleep(parameters.get("duration", 0.7))
            robot.stop()
        elif command == "stop":
            robot.stop()
        elif command == "dance":
            for _ in range(2):
                robot.left(0.5)
                await asyncio.sleep(0.5)
                robot.right(0.5)
                await asyncio.sleep(0.5)
            robot.stop()
        elif command == "describe" or command == "none":
            pass  # No action needed, just capture image
        else:
            logger.warning(f"Unknown command: {command}")
            robot.stop()  # Stop on unknown command
        logger.info(f"Executed command: {command}")
        return True
    except Exception as e:
        logger.error(f"Command execution error: {e}")
        robot.stop()  # Ensure robot stops on error
        return False

# 이미지 캡처
async def capture_image():
    if not camera:
        logger.error("Camera not initialized.")
        return None
    try:
        image = camera.value
        if image is not None:
             return base64.b64encode(bgr8_to_jpeg(image)).decode('utf-8')
        logger.warning("Camera returned no image.")
        return None
    except Exception as e:
        logger.error(f"Image capture error: {e}")
        return None


# 웹소켓 핸들러
async def websocket_handler(websocket, path):
    logger.info("WebSocket connection established")
    try:
        async for message in websocket:
            data = json.loads(message)
            command = data.get("command", "none")
            parameters = data.get("parameters", {})
            await handle_command(command, parameters)  # Execute the command
            image_base64 = await capture_image()
            if image_base64:
                await websocket.send(json.dumps({"image": image_base64}))
            else:
                logger.error("Image capture failed, sending empty response")
                await websocket.send(json.dumps({"image": ""})) # Send empty image if capture fails.
    except websockets.exceptions.ConnectionClosedOK:
        logger.info("WebSocket connection closed normally")
    except Exception as e:
        logger.error(f"WebSocket error: {e}")
#     finally:
#         if robot:
#             robot.stop()
#         logger.info("WebSocket connection closed")

# 메인 함수
async def main():
    server = await websockets.serve(websocket_handler, "0.0.0.0", WEBSOCKET_PORT)
    logger.info(f"WebSocket server running on port {WEBSOCKET_PORT}")
    await asyncio.Future()  # Run forever

if __name__ == "__main__":
    loop = asyncio.get_event_loop()
    try:
        loop.run_until_complete(main())
    except KeyboardInterrupt:
        logger.info("Shutting down...")
        if robot:
            robot.stop()
#     finally:
#         loop.close()

INFO:__main__:JetBot initialized successfully.


RuntimeError: This event loop is already running

INFO:__main__:WebSocket server running on port 8766
INFO:__main__:WebSocket connection established
INFO:__main__:Executed command: none
INFO:__main__:WebSocket connection established
INFO:__main__:Executed command: describe
INFO:__main__:WebSocket connection established
INFO:__main__:Executed command: none
INFO:__main__:WebSocket connection established
INFO:__main__:Executed command: describe
INFO:__main__:WebSocket connection established
INFO:__main__:Executed command: none
INFO:__main__:WebSocket connection established
INFO:__main__:Executed command: describe
INFO:__main__:WebSocket connection established
INFO:__main__:Executed command: none
INFO:__main__:WebSocket connection established
INFO:__main__:Executed command: describe


In [1]:
import asyncio
import websockets
import json
import logging
from jetbot import Robot, Camera, bgr8_to_jpeg
import base64
import time

# --- Logging Setup ---
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')  # Improved logging format
logger = logging.getLogger(__name__)

# --- Configuration ---
WEBSOCKET_PORT = 8766
CAMERA_WIDTH = 300
CAMERA_HEIGHT = 300
RECONNECT_DELAY = 5  # Seconds to wait before attempting to reconnect

# --- JetBot Initialization ---
try:
    robot = Robot()
    camera = Camera.instance(width=CAMERA_WIDTH, height=CAMERA_HEIGHT)
    logger.info("JetBot initialized successfully.")
except Exception as e:
    logger.error(f"JetBot initialization failed: {e}")
    robot = None
    camera = None

# --- Command Handling ---
async def handle_command(command, parameters=None):
    parameters = parameters or {}
    if not robot:
        logger.error("Robot not initialized.")
        return False

    try:
        if command == "forward":
            robot.forward(parameters.get("speed", 0.4))
            await asyncio.sleep(parameters.get("duration", 1.0))
            robot.stop()
        elif command == "backward":
            robot.backward(parameters.get("speed", 0.4))
            await asyncio.sleep(parameters.get("duration", 1.0))
            robot.stop()
        elif command == "left":
            robot.left(parameters.get("speed", 0.3))
            await asyncio.sleep(parameters.get("duration", 0.7))
            robot.stop()
        elif command == "right":
            robot.right(parameters.get("speed", 0.3))
            await asyncio.sleep(parameters.get("duration", 0.7))
            robot.stop()
        elif command == "stop":
            robot.stop()
        elif command == "dance":
            for _ in range(2):
                robot.left(0.5)
                await asyncio.sleep(0.5)
                robot.right(0.5)
                await asyncio.sleep(0.5)
            robot.stop()
        elif command == "describe" or command == "none":
            pass  # No action needed, just capture the image
        else:
            logger.warning(f"Unknown command: {command}")
            robot.stop()  # Always stop on an unknown command
        logger.info(f"Executed command: {command}")
        return True
    except Exception as e:
        logger.error(f"Command execution error: {e}")
        robot.stop()  # Ensure the robot stops on *any* error during command execution
        return False

# --- Image Capture ---
async def capture_image():
    if not camera:
        logger.error("Camera not initialized.")
        return None
    try:
        image = camera.value
        if image is not None:
            return base64.b64encode(bgr8_to_jpeg(image)).decode('utf-8')
        logger.warning("Camera returned no image.")
        return None
    except Exception as e:
        logger.error(f"Image capture error: {e}")
        return None

# --- WebSocket Handler ---
async def websocket_handler(websocket, path):
    logger.info("WebSocket connection established")
    while True:  # Continuous loop to handle reconnections
        try:
            async for message in websocket:
                data = json.loads(message)
                command = data.get("command", "none")
                parameters = data.get("parameters", {})
                await handle_command(command, parameters)
                image_base64 = await capture_image()
                if image_base64:
                    await websocket.send(json.dumps({"image": image_base64}))
                else:
                    logger.error("Image capture failed, sending empty response")
                    await websocket.send(json.dumps({"image": ""}))

        except websockets.exceptions.ConnectionClosedError as e:
            logger.warning(f"WebSocket connection closed: {e}.  Attempting to reconnect...")
            if robot:
                robot.stop()
            await asyncio.sleep(RECONNECT_DELAY)  # Wait before reconnecting
        except websockets.exceptions.ConnectionClosedOK:
            logger.info("WebSocket connection closed normally.")
            if robot:
                robot.stop()
            break  # Exit loop on normal close
        except Exception as e:
            logger.error(f"WebSocket error: {e}")
            if robot:
                robot.stop() # Stop the robot
            await asyncio.sleep(RECONNECT_DELAY) # Wait before reconnect
        #except Exception as e: #Removed this block as all exception are handled
        #    logger.error(f"WebSocket error: {e}")
        #    break  # Break on other errors

# --- Main Function ---
async def main():
    while True: # Continuous loop for the server
        try:
            async with websockets.serve(websocket_handler, "0.0.0.0", WEBSOCKET_PORT):
                logger.info(f"WebSocket server running on port {WEBSOCKET_PORT}")
                await asyncio.Future()  # Run forever *inside* the `with` statement
        except OSError as e:
            if "Address already in use" in str(e):
                logger.error(f"Address already in use: {e}.  Retrying in {RECONNECT_DELAY} seconds...")
                await asyncio.sleep(RECONNECT_DELAY)
            else:
                logger.error(f"Server encountered an OSError: {e}") # Other OS errors are unlikely
                break # If not address in use, do not retry
        except Exception as e: # Generic exception
            logger.error(f"Server encountered an error: {e}")
            break # Break on other errors
if __name__ == "__main__":
    loop = asyncio.get_event_loop()
    try:
        loop.run_until_complete(main())
    except KeyboardInterrupt:
        logger.info("Shutting down...")
        if robot:
            robot.stop()
    finally:
        loop.close()

2025-03-20 01:54:40,456 - INFO - JetBot initialized successfully.


RuntimeError: Cannot close a running event loop

2025-03-20 01:54:40,879 - INFO - WebSocket server running on port 8766
2025-03-20 01:55:46,374 - INFO - WebSocket connection established
2025-03-20 01:55:46,408 - INFO - Executed command: none
2025-03-20 01:55:49,137 - INFO - WebSocket connection established
2025-03-20 01:55:49,146 - INFO - Executed command: none
2025-03-20 01:55:54,662 - INFO - WebSocket connection established
2025-03-20 01:55:54,677 - INFO - Executed command: none
2025-03-20 01:55:59,576 - INFO - WebSocket connection established
2025-03-20 01:55:59,595 - INFO - Executed command: stop
2025-03-20 01:56:25,675 - INFO - WebSocket connection established
2025-03-20 01:56:26,000 - INFO - Executed command: none
2025-03-20 01:56:29,059 - INFO - WebSocket connection established
2025-03-20 01:56:29,070 - INFO - Executed command: none
2025-03-20 01:56:34,276 - INFO - WebSocket connection established
2025-03-20 01:56:34,291 - INFO - Executed command: none
2025-03-20 01:56:38,272 - INFO - WebSocket connection established
2025-03-2

In [1]:
import asyncio
import websockets
import json
import logging
from jetbot import Robot, Camera, bgr8_to_jpeg
import base64
import time

# --- Logging Setup ---
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# --- Configuration ---
WEBSOCKET_PORT = 8766  # WebSocket port
CAMERA_WIDTH = 300     # Camera resolution
CAMERA_HEIGHT = 300
RECONNECT_DELAY = 5  # Seconds to wait before reconnecting

# --- JetBot Initialization ---
try:
    robot = Robot()
    camera = Camera.instance(width=CAMERA_WIDTH, height=CAMERA_HEIGHT)
    logger.info("JetBot initialized successfully.")
except Exception as e:
    logger.error(f"JetBot initialization failed: {e}")
    robot = None
    camera = None

# --- Command Handling ---
async def handle_command(command, parameters=None):
    """
    Executes commands received from the client (FastAPI server).
    """
    parameters = parameters or {}
    if not robot:
        logger.error("Robot not initialized.")
        return False

    try:
        if command == "forward":
            robot.forward(parameters.get("speed", 0.4))
            await asyncio.sleep(parameters.get("duration", 1.0))
            robot.stop()
        elif command == "backward":
            robot.backward(parameters.get("speed", 0.4))
            await asyncio.sleep(parameters.get("duration", 1.0))
            robot.stop()
        elif command == "left":
            robot.left(parameters.get("speed", 0.3))
            await asyncio.sleep(parameters.get("duration", 0.7))
            robot.stop()
        elif command == "right":
            robot.right(parameters.get("speed", 0.3))
            await asyncio.sleep(parameters.get("duration", 0.7))
            robot.stop()
        elif command == "stop":
            robot.stop()
        elif command == "dance":
            for _ in range(2):
                robot.left(0.5)
                await asyncio.sleep(0.5)
                robot.right(0.5)
                await asyncio.sleep(0.5)
            robot.stop()
        elif command == "describe" or command == "none":
            pass  # No action needed, just capture the image
        else:
            logger.warning(f"Unknown command: {command}")
            robot.stop()  # Always stop on an unknown command
        logger.info(f"Executed command: {command}")
        return True
    except Exception as e:
        logger.error(f"Command execution error: {e}")
        robot.stop()  # Ensure the robot stops on *any* error
        return False

# --- Image Capture ---
async def capture_image():
    """
    Captures an image from the JetBot's camera and returns it as a base64 string.
    """
    if not camera:
        logger.error("Camera not initialized.")
        return None
    try:
        image = camera.value
        if image is not None:
             return base64.b64encode(bgr8_to_jpeg(image)).decode('utf-8')
        logger.warning("Camera returned no image.")
        return None
    except Exception as e:
        logger.error(f"Image capture error: {e}")
        return None

# --- WebSocket Handler ---
async def websocket_handler(websocket, path):
    """
    Handles WebSocket connections and messages.  Includes reconnection logic.
    """
    logger.info("WebSocket connection established")
    while True:  # Continuous loop for reconnection
        try:
            async for message in websocket:
                data = json.loads(message)
                command = data.get("command", "none")
                parameters = data.get("parameters", {})
                await handle_command(command, parameters)
                image_base64 = await capture_image()
                if image_base64:
                    await websocket.send(json.dumps({"image": image_base64}))
                    logger.info(f"Image sent (first 100 chars): {image_base64[:100]}...")
                else:
                    logger.error("Image capture failed, sending empty response")
                    await websocket.send(json.dumps({"image": ""}))

        except websockets.exceptions.ConnectionClosedError as e:
            logger.warning(f"WebSocket connection closed: {e}.  Attempting to reconnect...")
            if robot:
                robot.stop()  # Stop the robot on disconnection
            await asyncio.sleep(RECONNECT_DELAY)  # Wait before reconnecting
        except websockets.exceptions.ConnectionClosedOK:
            logger.info("WebSocket connection closed normally.")
            if robot:
                robot.stop()
            break  # Exit loop on normal close
        except Exception as e:
            logger.error(f"WebSocket error: {e}")
            if robot:
                robot.stop()
            await asyncio.sleep(RECONNECT_DELAY)

# --- Main Function ---
async def main():
    """
    Starts the WebSocket server and keeps it running.  Handles server restarts.
    """
    while True:  # Continuous loop for server restart
        try:
            async with websockets.serve(websocket_handler, "0.0.0.0", WEBSOCKET_PORT):
                logger.info(f"WebSocket server running on port {WEBSOCKET_PORT}")
                await asyncio.Future()  # Keep the server running indefinitely
        except OSError as e:
            if "Address already in use" in str(e):
                logger.error(f"Address already in use: {e}.  Retrying in {RECONNECT_DELAY} seconds...")
                await asyncio.sleep(RECONNECT_DELAY) # Wait before retrying
            else:
                logger.error(f"Server encountered an OSError: {e}")
                break  # Don't retry for other OSErrors
        except Exception as e:
            logger.error(f"Server encountered an error: {e}")
            break # Don't retry for other errors

# --- Run the Server ---
if __name__ == "__main__":
    loop = asyncio.get_event_loop()
    try:
        loop.run_until_complete(main())
    except KeyboardInterrupt:
        logger.info("Shutting down...")
        if robot:
            robot.stop()  # Stop the robot on shutdown
    finally:
        loop.close()  # Close the event loop

2025-03-20 08:21:48,358 - INFO - JetBot initialized successfully.


RuntimeError: Cannot close a running event loop

2025-03-20 08:21:48,786 - INFO - WebSocket server running on port 8766
2025-03-20 08:22:11,359 - INFO - WebSocket connection established
2025-03-20 08:22:11,366 - INFO - Executed command: none
2025-03-20 08:22:11,378 - INFO - Image sent (first 100 chars): /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQECAgICAgQDAgICAgUEBAMEBgUGBgYFBgYGBwkIBgcJBwYGCAsICQoK...
2025-03-20 08:22:19,656 - INFO - WebSocket connection established
2025-03-20 08:22:19,671 - INFO - Executed command: describe
2025-03-20 08:22:19,684 - INFO - Image sent (first 100 chars): /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQECAgICAgQDAgICAgUEBAMEBgUGBgYFBgYGBwkIBgcJBwYGCAsICQoK...
2025-03-20 08:22:34,092 - INFO - WebSocket connection established
2025-03-20 08:22:34,109 - INFO - Executed command: none
2025-03-20 08:22:34,119 - INFO - Image sent (first 100 chars): /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQECAgICAgQDAgICAgUEBAMEBgUGBgYFBgYGBwkIBgcJBwYGCAsICQoK...
2025-03-20 08:22:36,865 - INFO - WebSocket connection esta